In [1]:
import scipy.io
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torchvision.transforms import ToTensor
from sklearn.model_selection import train_test_split
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

In [2]:
data = scipy.io.loadmat("../data/ex4data1.mat")
raw_X = data['X']
raw_y = data['y']
raw_y[raw_y == 10] = 0

x_train, x_test, y_train, y_test = train_test_split(raw_X, raw_y, test_size=0.2, random_state=42)

In [3]:
class CustomDataset(Dataset):
    def __init__(self, features, labels, transform=None):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.int64).squeeze()
        self.transform = transform

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        feature = self.features[idx]
        label = self.labels[idx]
        if self.transform:
            feature = self.transform(feature)
        return feature, label


train_dataset = CustomDataset(x_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataset = CustomDataset(x_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)


for X, y in train_loader:
    print(f"Feature batch shape: {X.size()}")
    print(f"Labels batch shape: {y.size()}")
    break

Feature batch shape: torch.Size([64, 400])
Labels batch shape: torch.Size([64])


In [4]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(20*20, 512),
            nn.GELU(),
            nn.Dropout(0.5),
            nn.Linear(512, 256),
            nn.GELU(),
            nn.Dropout(0.5),
            nn.Linear(256, 10)
        )

        # 使用 Xavier 初始化权重
        for layer in self.linear_relu_stack:
            if isinstance(layer, nn.Linear):
                nn.init.xavier_normal_(layer.weight, gain=1.414)
                nn.init.zeros_(layer.bias)

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=400, out_features=512, bias=True)
    (1): GELU(approximate='none')
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=512, out_features=256, bias=True)
    (4): GELU(approximate='none')
    (5): Dropout(p=0.5, inplace=False)
    (6): Linear(in_features=256, out_features=10, bias=True)
  )
)


In [5]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
scheduler = StepLR(optimizer, step_size=10, gamma=0.9)

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for batch_index, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)

        inputs = inputs.view(-1, 1, 20, 20)
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (batch_index + 1) % 20 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{batch_index+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

Epoch [1/10], Step [20/63], Loss: 1.1177
Epoch [1/10], Step [40/63], Loss: 0.6973
Epoch [1/10], Step [60/63], Loss: 0.5578
Epoch [2/10], Step [20/63], Loss: 0.5914
Epoch [2/10], Step [40/63], Loss: 0.2852
Epoch [2/10], Step [60/63], Loss: 0.3296
Epoch [3/10], Step [20/63], Loss: 0.3275
Epoch [3/10], Step [40/63], Loss: 0.4685
Epoch [3/10], Step [60/63], Loss: 0.5012
Epoch [4/10], Step [20/63], Loss: 0.3643
Epoch [4/10], Step [40/63], Loss: 0.2039
Epoch [4/10], Step [60/63], Loss: 0.2466
Epoch [5/10], Step [20/63], Loss: 0.1841
Epoch [5/10], Step [40/63], Loss: 0.3512
Epoch [5/10], Step [60/63], Loss: 0.0971
Epoch [6/10], Step [20/63], Loss: 0.1933
Epoch [6/10], Step [40/63], Loss: 0.0992
Epoch [6/10], Step [60/63], Loss: 0.2668
Epoch [7/10], Step [20/63], Loss: 0.1363
Epoch [7/10], Step [40/63], Loss: 0.1311
Epoch [7/10], Step [60/63], Loss: 0.2324
Epoch [8/10], Step [20/63], Loss: 0.1674
Epoch [8/10], Step [40/63], Loss: 0.2333
Epoch [8/10], Step [60/63], Loss: 0.2096
Epoch [9/10], St

In [7]:
model.eval()
test_loss, correct = 0, 0
size = len(test_loader.dataset)
num_batches = len(test_loader)
with torch.no_grad():
    for X, y in test_loader:
        X, y = X.to(device), y.to(device)
        X = X.view(-1, 1, 20, 20)
        pred = model(X)
        test_loss += criterion(pred, y).item()
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n  Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

Test Error: 
  Accuracy: 94.6%, Avg loss: 0.218327 

